In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile,isdir, join
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D, Dropout,Activation,MaxPooling1D,Flatten
from keras.optimizers import RMSprop
from scipy.fftpack import fft

In [ ]:

allX = []
allY = []
maxi=0.
mini=0.
count=0


path0="./BASE DE DATOS"  #estos son datos de encefalogramas 

carpetas = [ f for f in listdir(path0) if isdir(join(path0,f))]
carpetas_f = [ k for k in carpetas if 'Data_F' in k]
carpetas_n = [ k for k in carpetas if 'Data_N' in k]
print(carpetas)
print(carpetas_f)
print(carpetas_n)
for d in carpetas_f:
    path = join(path0,d)
    files = [ f for f in listdir(path)]
    for f in files:
        serie = []
        with open(join(path,f), newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in spamreader:
                serie.append(float(row[0]))
        fserie = np.abs(fft(serie))
        allX.append(fserie)
        allY.append(0)
        max_tmp=max(serie)
        if (max_tmp > maxi):
            maxi = max_tmp
        min_tmp=min(serie)
print(len(allX))

In [ ]:
maxi=0.
mini=0.
for d in carpetas_n:
    path = join(path0,d)
    files = [ f for f in listdir(path)]
    for f in files:
        serie = []
        with open(join(path,f), newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in spamreader:
                serie.append(float(row[0]))
        fserie = np.abs(fft(serie))
        allX.append(fserie)
        allY.append(1)
        max_tmp=max(serie)
        if (max_tmp > maxi):
            maxi = max_tmp
        min_tmp=min(serie)
        if (min_tmp < mini):
            mini = min_tmp
        count+=1
print(count)
allX = np.array(allX)
allY = np.array(allY)
allX = (allX - mini)/(maxi - mini)
x_train, x_test, y_train, y_test = train_test_split(allX, allY, test_size=0.3, random_state=1)
n_patterns = len(x_train)
seq_length = 10240
x_train = np.reshape(x_train, (n_patterns, seq_length, 1))
n_patterns_test = len(x_test)
x_test = np.reshape(x_test, (n_patterns_test, seq_length, 1))
print(n_patterns_test)
print(x_test[2,:,0])
print("----")
print(y_test)
print("----")

In [ ]:
model = Sequential()

#primera capa: convolucional de 1D con 10 filtros y 30 de ventana
model.add(Conv1D(10, 30, input_shape=(10240,1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

#segunda capa convolucional 1D ignorada
# model.add(Conv1D(10,  15))
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size=2))

#segunda capa convolucional 1D 
model.add(Conv1D(20, 15))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

#aplanamos los datos de forma que pasemos de convolucional a capa densa
model.add(Flatten())

#tercera capa densa de 64 neuronas 
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.2))

#capa de salida binaria
model.add(Dense(1))
model.add(Activation('sigmoid'))

#compilamos el modelo con la funcion de costo de cross entropy para binaria y el optimizador "RMSprop" 
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./electroGraph1D', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
history = model.fit(x_train, y_train,batch_size=100,epochs=30,verbose=1,callbacks=[tbCallBack],validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=1)
print(score)